In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import math

In [2]:
class TokenEmbedding(nn.Embedding):
    # 将输入的词汇表转化为指定维度的Embedding
    def __init__(self,vocab_size,d_model):
        super().__init__(num_embeddings=vocab_size,embedding_dim=d_model,padding_idx=1)

$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i / d_{\text{model}}}}\right)$

$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i / d_{\text{model}}}}\right)$

In [ ]:
class PositionalEmbedding(nn.Module):
    def __init__(self,d_model,max_len,device):
        super().__init__()
        self.device=torch.device(device)
        self.encoding=torch.zeros(max_len,d_model,device=self.device)
        self.encoding.requires_grad=False
        pos=torch.arange(0,max_len,device=self.device,dtype=torch.float32)
        pos=pos.unsequeeze(dim=1) # pos的大小为(max_len,1)
        # unsequeezed的作用是在张量的指定位置插入大小为1的维度
        _zi=torch.arange(0,d_model,step=2,dtype=torch.float32,device=self.device)
        self.encoding[:,0::2]=torch.sin(pos/(1000**_zi/d_model))
        self.encoding[:,1::2]=torch.cos(pos/(1000**_zi/d_model))
        # self.encoding的大小为(max_len,d_model)
    def forward(self,x):
        batch_size,seq_len=x.size()
        return self.encoding[:seq_len,:] # return 的大小是(seq_len,d_model)

In [ ]:
class TransformerEmbedding(nn.Module):
    def __init__(self,vocab_size,d_model,max_len,drop_prod,device):
        super().__init__()
        pass